# Instalación

Las herramientas necesarias para entrenar el modelo se obtienen de nuestro repositorio `diaxi-training-tools`.

In [ ]:
%%bash
rm -rf tools
git clone -q 'https://github.com/epassaro/diaxi-training-tools' tools
pip install -q -r tools/requirements.txt

> **ADVERTENCIA:** se va a reiniciar el entorno de ejecución, es normal, ignorar los mensajes de error y continuar ejecutando el resto de la notebook.

In [ ]:
get_ipython().kernel.do_shutdown(True)

<br>

# Probando el modelo `PrimaLayout`

Elegimos el modelo [`PrimaLayout`](https://layout-parser.readthedocs.io/en/latest/notes/modelzoo.html#model-catalog) como punto de partida para hacer *transfer learning* por estar entrenado con un set de datos de imágenes de diarios y revistas.  Es un modelo tipo [Mask R-CNN](https://wiki.math.uwaterloo.ca/statwiki/index.php?title=Mask_RCNN) y fue hecho con la librería de [**Detectron2**](https://github.com/facebookresearch/detectron2) de Facebook AI.

<br>

Vamos a descargar la configuración y los pesos del modelo para ver como se desempeña.

In [ ]:
%%bash
mkdir -p model
curl -sL 'https://www.dropbox.com/s/yc92x97k50abynt/config.yaml?dl=1' -o model/config.yaml
curl -sL 'https://www.dropbox.com/s/h7th27jfv19rxiy/model_final.pth?dl=1' -o model/model_final.pth

<br>

Leemos la configuración y a creamos un objeto de la clase `DefaultPredictor`.

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor


cfg = get_cfg()
cfg.merge_from_file("model/config.yaml")
cfg.MODEL.WEIGHTS = "model/model_final.pth"

predictor = DefaultPredictor(cfg)

<br>

Descargamos una imagen de prueba y procedemos a hacer la inferencia.

In [ ]:
%%bash
curl -sL 'https://www.lavoz.com.ar/resizer/xuZbQG2Eksz9cu5TUO-h725zcPI=/1023x1428/smart/cloudfront-us-east-1.images.arcpublishing.com/grupoclarin/LFOKK4SCLNDPHGITAJ35EGKWKE.jpg' -o sample_data/image.jpg

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode


image = cv2.imread("sample_data/image.jpg")
metadata = MetadataCatalog.get("prima_layout")
metadata.thing_classes = ["UnknownClass", "TextRegion", "ImageRegion", "TableRegion", "MathsRegion", "SeparatorRegion", "OtherRegion"]

v = Visualizer(image[:, :, ::-1],
               metadata=metadata,
               scale=0.5,
               instance_mode=ColorMode.IMAGE_BW
              )

output = predictor(image)
out = v.draw_instance_predictions(output["instances"].to("cpu"))

cv2_imshow(out.get_image()[:, :, ::-1])

<br>

El resultado es muy bueno. Ahora queremos reentrenarlo para que detecte las clases de nuestro problema.


<br>

# Reentrenando el modelo

Para reentrenar el modelo utilizamos un dataset de 497 imágenes en formato [COCO](https://cocodataset.org/). Del total de imágenes, 297 provienen de la fase de etiquetado colaborativo, mientras que 200 fueron agregadas por nosotros posteriormente para aumentar el tamaño de la muestra.

Además, se calcularon las *máscaras de segmentación* que el modelo requiere, entre varios cambios que fue necesario hacer para ajustarse al formato requerido.

<br>

> *Para ver los detalles de la creación del dataset referirse al [repositorio de entrenamiento](https://github.com/epassaro/diaxi-training-tools).*


In [ ]:
%%bash
rm -rf dataset dataset-diaxi-coco.zip
curl -sL 'http://xmm-newton.fcaglp.unlp.edu.ar/assets/dataset-diaxi-coco497.zip' -o dataset-diaxi-coco.zip
unzip -q dataset-diaxi-coco.zip -d dataset && rm dataset-diaxi-coco.zip

<br>

Ahora dividimos en set de entrenamiento (85%) y validación (15%).

In [ ]:
%%bash
pyodi coco random-split dataset/result.json dataset/result --val-percentage 0.15

<br>

Y los registramos.

In [ ]:
from detectron2.data.datasets import register_coco_instances


register_coco_instances("diaxi_train", {}, "dataset/result_train.json", "dataset")
register_coco_instances("diaxi_val", {}, "dataset/result_val.json", "dataset")

<br>

Hacemos los cambios necesarios en la configuración del modelo para hacer *transfer learning*:

- Partir desde los pesos y la configuración del modelo anterior
- Setear el número máximo de iteraciones
- Setear el número de iteraciones entre cada llamado al evaluador
- Setear el número de iteraciones entre cada *checkpoint*
- Ajustar el número de clases

In [ ]:
import os
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("model/config.yaml")
cfg.MODEL.WEIGHTS = "model/model_final.pth"
cfg.DATASETS.TRAIN = ("diaxi_train",)
cfg.DATASETS.TEST = ("diaxi_val",)
cfg.OUTPUT_DIR = "output"
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []
cfg.SOLVER.CHECKPOINT_PERIOD = 200
cfg.TEST.EVAL_PERIOD = 200
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 7

_ = os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

<br>

Creamos la clase `CustomTrainer` y agregamos el código necesario para llamar al evaluador.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data import transforms as T

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator("diaxi_val", cfg, True, output_folder)

    # No hay suficiente VRAM para hacer data augmentation :()
    # def build_train_loader(cls, cfg):
    #     mapper = DatasetMapper(cfg, is_train=True, augmentations=[T.RandomFlip(prob=0.5, horizontal=True, vertical=False)])
    #     return build_detection_train_loader(cfg, mapper=mapper)

<br>

Iniciamos el *dashboard* de Tensorflow para monitorear la evolución de la métrica y entrenamos el modelo.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output/inference

In [ ]:
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)

trainer.train()

<br>

Luego de examinar cuidadosamente las métricas en *TensorBoard*, guardamos el modelo que vamos a implementar en el software. En nuestro caso elegimos el *checkpoint* de 2600 iteraciones.

In [ ]:
with open("output/config.yaml", "w") as f:
   f.write(trainer.cfg.dump())

In [ ]:
%%bash
mkdir -p download
cp output/config.yaml download/
cp output/metrics.json download/
cp output/model_0002599.pth download/model_final.pth

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("download/config.yaml")
cfg.MODEL.WEIGHTS = "https://github.com/epassaro/diaxi-training-tools/releases/download/v0.1/model_final.pth"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8
cfg.MODEL.DEVICE = "cpu"

# Sobreescribimos
with open("download/config.yaml", "w") as f:
   f.write(cfg.dump())

In [ ]:
%%bash
tar -zcvf model_final.tar.gz model_final